# Data Science Technology and Systems PG (11523)
# Final Project | Semester 2, 2025

### Natalia Andrea Cubillos Villegas
### Student ID: U3246979

# Problem: Predicting Airplane Delays

The goals of this notebook are:
- Process and create a dataset from downloaded ZIP files
- Exploratory data analysis (EDA)
- Establish a baseline model and improve it

## Introduction to business scenario
You work for a travel booking website that is working to improve the customer experience for flights that were delayed. The company wants to create a feature to let customers know if the flight will be delayed due to weather when the customers are booking the flight to or from the busiest airports for domestic travel in the US. 

You are tasked with solving part of this problem by leveraging machine learning to identify whether the flight will be delayed due to weather. You have been given access to the a dataset of on-time performance of domestic flights operated by large air carriers. You can use this data to train a machine learning model to predict if the flight is going to be delayed for the busiest airports.

### Dataset
The provided dataset contains scheduled and actual departure and arrival times reported by certified US air carriers that account for at least 1 percent of domestic scheduled passenger revenues. The data was collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS). The dataset contains date, time, origin, destination, airline, distance, and delay status of flights for flights between 2014 and 2018.
The data are in 60 compressed files, where each file contains a CSV for the flight details in a month for the five years (from 2014 - 2018). The data can be downloaded from this [link](https://ucstaff-my.sharepoint.com/:f:/g/personal/ibrahim_radwan_canberra_edu_au/EhWeqeQsh-9Mr1fneZc9_0sBOBzEdXngvxFJtAlIa-eAgA?e=8ukWwa). Please download the data files and place them on a relative path. Dataset(s) used in this assignment were compiled by the Office of Airline Information, Bureau of Transportation Statistics (BTS), Airline On-Time Performance Data, available with the following [link](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ). 

# Step 1: Prepare the environment 

Use one of the labs which we have practised on with the Amazon Sagemakers where you perform the following steps:
1. Start a lab.
2. Create a notebook instance and name it "oncloudproject".

**Unfortunately, I was not able to create a new notebook instance because the AWS platform indicated that access was denied for my account.**
**I attempted to modify my role and increase permissions, but this was not possible.**

**I also tried to create a new session through Python; however, when checking Access Management >> Users, I received the following message:**

***“Required: An IAM user is an identity with long-term credentials that is used to interact with AWS in an account.”***

**Moreover, I tried to access SageMaker from one of our labs. Although I successfully logged in, I encountered the same “Access Denied” message when attempting to implement the Linear Learner Estimator.**

**Given these limitations, I decided to complete the tasks required for this part of the assignment locally using Jupyter Notebook instead.**

# Step 2: Build and evaluate simple models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use linear learner estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 

Note: You are required to perform the above steps on the two combined datasets separatey and to comments on the difference.

In [3]:
# Loading the necessary libraries
import pandas as pd
import numpy as np
import os

# Setting the random seed for reproducibility
np.random.seed(42)

# Defining the base path for data files
base_path = "Final_Project"
data_v1 = pd.read_csv(f"{base_path}/combined_csv_v1.csv")
data_v2 = pd.read_csv(f"{base_path}/combined_csv_v2.csv")

### Working with data version 1

In [4]:
# Splitting data into traiing, validation, and test sets (70%, 15%, 15%)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_v1, test_size=0.15, random_state=42, stratify=data_v1['target'])
train_data, val_data = train_test_split(train_data, test_size=0.1765, random_state=42, stratify=train_data['target'])  # 0.1765 x 0.85 = 0.15

# Displaying the shapes of the datasets
print(f"Train data shape - Data Version 1: {train_data.shape}")
print(f"Validation data shape - Data Version 1: {val_data.shape}")
print(f"Test data shape - Data Version 1: {test_data.shape}")

# Tackling class imbalance using SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train = train_data.drop('target', axis=1)
y_train = train_data['target']
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(f'Original dataset shape - Data Version 1: {y_train.value_counts().to_dict()}')
print(f'Resampled dataset shape - Data Version 1: {y_resampled.value_counts().to_dict()}')

Train data shape - Data Version 1: (1144871, 94)
Validation data shape - Data Version 1: (245380, 94)
Test data shape - Data Version 1: (245339, 94)
Original dataset shape - Data Version 1: {0.0: 904547, 1.0: 240324}
Resampled dataset shape - Data Version 1: {1.0: 904547, 0.0: 904547}


In [5]:
# Using Logistic Regression as a baseline model instead of Linear Learner Estimator from AWS SageMaker
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Training the Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_resampled, y_resampled)

# Making predictions on the validation set
y_val_pred = logistic_model.predict(val_data.drop('target', axis=1))

# Evaluating the model
print(confusion_matrix(val_data['target'], y_val_pred))
print(classification_report(val_data['target'], y_val_pred))

[[131525  62347]
 [ 27305  24203]]
              precision    recall  f1-score   support

         0.0       0.83      0.68      0.75    193872
         1.0       0.28      0.47      0.35     51508

    accuracy                           0.63    245380
   macro avg       0.55      0.57      0.55    245380
weighted avg       0.71      0.63      0.66    245380



/opt/anaconda3/envs/spyder/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Improving performance using CVTuning with GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_resampled, y_resampled)
print(f"Best parameters from GridSearchCV: {grid_search.best_params_}")
# Making predictions with the best model
best_model = grid_search.best_estimator_
y_val_pred_best = best_model.predict(val_data.drop('target', axis=1))
# Evaluating the best model
print(confusion_matrix(val_data['target'], y_val_pred_best))
print(classification_report(val_data['target'], y_val_pred_best))

### Working with data version 2

In [ ]:
# Splitting data into traiing, validation, and test sets (70%, 15%, 15%)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_v2, test_size=0.15, random_state=42, stratify=data_v2['target'])
train_data, val_data = train_test_split(train_data, test_size=0.1765, random_state=42, stratify=train_data['target'])  # 0.1765 x 0.85 = 0.15

# Displaying the shapes of the datasets
print(f"Train data shape - Data V2: {train_data.shape}")
print(f"Validation data shape - Data V2: {val_data.shape}")
print(f"Test data shape - Data V2: {test_data.shape}")

# Tackling class imbalance using SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train = train_data.drop('target', axis=1)
y_train = train_data['target']
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(f'Original dataset shape - Data V2: {y_train.value_counts().to_dict()}')
print(f'Resampled dataset shape - Data V2: {y_resampled.value_counts().to_dict()}')

In [ ]:
# Using Logistic Regression as a baseline model instead of Linear Learner Estimator from AWS SageMaker
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Training the Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_resampled, y_resampled)

# Making predictions on the validation set
y_val_pred = logistic_model.predict(val_data.drop('target', axis=1))

# Evaluating the model
print(confusion_matrix(val_data['target'], y_val_pred))
print(classification_report(val_data['target'], y_val_pred))

# Step 3: Build and evaluate ensembe models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use xgboost estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 
6. write down your observation on the difference between the performance of using the simple and ensemble models.
Note: You are required to perform the above steps on the two combined datasets separatey.

In [4]:
# Write the code here and add cells as you need

In [5]:
# Write the final comments here and turn the cell type into markdown